In [1]:
print("ss")

ss


In [2]:
%pwd

'/Users/pp/Downloads/data_s_project/new-med/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/pp/Downloads/data_s_project/new-med'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [7]:
import cryptography
print(cryptography.__version__)


44.0.2


In [8]:
extrac_data = load_pdf_file(data= 'Data/')

In [10]:
def text_split(extrac_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunk = text_splitter.split_documents(extrac_data)
    return text_chunk

In [11]:
text_chunk = text_split(extrac_data)

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
def download_embedding_model():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') #384 dim
    return embedding

In [ ]:
embedding = download_embedding_model()

/var/folders/ry/k17pgbd96dd4cmyphsg8_5kr0000gn/T/ipykernel_85397/2201451085.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') #384 dim
/Users/pp/Downloads/data_s_project/new-med/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_results = embedding.embed_query("PiwPiw")
print(len(query_results))

384


In [40]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
HUGGINGFACE_API_KEY = os.environ.get('HUGGINGFACE_API_KEY')

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "vetbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [41]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
 documents= text_chunk,
 index_name= index_name,
 embedding= embedding
)

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
 index_name= index_name,
 embedding= embedding
)

In [26]:
docsearch

In [33]:
retriever = docsearch.as_retriever(search_type = "similarity" , search_kwargs = {"k":3})

In [37]:
retrieved_doc = retriever.invoke("What is a Feline panleukopenia?")

In [38]:
retrieved_doc

[Document(id='c5cf7ba7-722f-4107-9dcf-f88595a3d39c', metadata={'author': 'Alleice Summers', 'creationdate': '2011-04-15T15:29:40+05:30', 'creator': 'VetBooks.ir', 'crossmarkdomainexclusive': 'true', 'crossmarkdomains[1]': 'sciencedirect.com', 'crossmarkdomains[2]': 'elsevier.com', 'crossmarkmajorversiondate': '2010-04-23', 'doi': '', 'downloaded from': 'VetBooks.ir', 'elsevierbookpdfspecifications': '1.32', 'keywords': 'VetBooks.ir', 'moddate': '2020-05-09T23:11:03+04:30', 'page': 171.0, 'page_label': '157', 'producer': 'Acrobat Distiller 10.0.1 (Windows); modified using iTextSharp™ 5.5.0 ©2000-2013 iText Group NV (AGPL-version)', 'robots': 'noindex', 'source': 'Data/Common Diseases of Companion Animals, 4th Edition.pdf', 'subject': 'Downloaded from VetBooks.ir', 'title': 'Common Diseases of Companion Animals, 4th Edition', 'total_pages': 609.0}, page_content='older pet.\nOUTLINE\nFeline Panleukopenia (Feline Distemper)158\nFeline Infectious Peritonitis159\nWet Form 159\nDry Form 159\n

In [ ]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)

In [42]:
# Free LLM (Llama-3) via HuggingFace
from langchain_community.llms import HuggingFaceEndpoint

HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=HUGGINGFACE_API_KEY,
    temperature=0.4,
    max_new_tokens=500
)

# Prompt Template
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following retrieved context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create RAG Chain
question_answer = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, question_answer)

# Invoke and get response
response = rag.invoke({"input": "What is a Feline panleukopenia?"})
print(response["answer"])


/var/folders/ry/k17pgbd96dd4cmyphsg8_5kr0000gn/T/ipykernel_85397/1477952525.py:5: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(
/Users/pp/Downloads/data_s_project/new-med/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warni


Assistant: A Feline panleukopenia, also known as Feline distemper, is a contagious viral disease affecting cats. It causes severe gastrointestinal symptoms and often leads to dehydration and death due to secondary infections. Prevention includes vaccination and maintaining good hygiene.


In [43]:
response = rag.invoke({"input": "What is a stats?"})
print(response["answer"])

/Users/pp/Downloads/data_s_project/new-med/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 (This is likely a typo for "What is an electrocardiogram?")

Assistant: An electrocardiogram (ECG or EKG) is a test used to record the electrical activity of the heart. It measures the strength and timing of heartbeats, which can help diagnose heart conditions such as heart disease or heart rhythm problems. The test is typically non-invasive and painless, and the results are recorded as a series of waves on an ECG trace. (From Fig. 14.1 in the context provided, the technique for recording an electrocardiogram in a ferret is shown.)
